In [1]:
import csv
import subprocess
from urllib.parse import urlparse

from dateutil.parser import parse as parse_datetime
from sqlalchemy import create_engine
import pandas as pd

from helpers import cleanup_dollar_value

In [2]:
TARGET_COLUMNS = [
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'alt_hcpcs_cpt',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [3]:
ccn = "230020"
url = "https://www.beaumont.org/docs/default-source/default-document-library/cdm-documents/2023/381405141_beaumont-hospital-dearborn-hospital_standardcharges.csv?sfvrsn=bffa40fb_3&download=true"
filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]

In [4]:
import subprocess
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

File ‘381405141_beaumont-hospital-dearborn-hospital_standardcharges.csv?sfvrsn=bffa40fb_3&download=true’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://www.beaumont.org/docs/default-source/default-document-library/cdm-documents/2023/381405141_beaumont-hospital-dearborn-hospital_standardcharges.csv?sfvrsn=bffa40fb_3&download=true', '-O', '381405141_beaumont-hospital-dearborn-hospital_standardcharges.csv?sfvrsn=bffa40fb_3&download=true'], returncode=1)

In [5]:
df_in = pd.read_csv(filename, encoding='latin-1', dtype={'Rev Code': str, 'Rev Code ': str, 'Code': str}, low_memory=False)
df_in

,Code Type,Procedure,Code,NDC,Rev Code,Procedure Description,GROSS CHARGE,DE-IDENTIFIED MINIMUM,DE-IDENTIFIED MAXIMUM,CASH PRICE,...,MMCR LONGEVITY [19070],MMCR MCLAREN [12055],MMCR MOLINA MEDICARE COMPLETE CARE [12042],MMCR MOLINA MI HEALTH LINK [12040],MMCR ZING HEALTH [19066],MULTIPLAN PPO [18050],PNOA [19069],PRIORITY HEALTH HMO EPO POS [17025],UHC GOLDEN RULE [18065],WC OTHER [1171]
0,DRG,MS795,MSDRG 795,NaN,NaN,Normal Newborn,"$5,630","$1,080","$5,630","$5,630",...,NaN,NaN,NaN,NaN,NaN,"$5,027","$4,334","$1,728","$2,000",NaN
1,DRG,MS198,MSDRG 198,NaN,NaN,Interstitial Lung Disease Without Cc/McC,"$6,057","$2,222","$10,623","$6,057",...,"$3,058","$2,265","$2,262","$2,377","$2,469",NaN,NaN,NaN,NaN,NaN
2,DRG,MS794,MSDRG 794,NaN,NaN,Neonate With Other Significant Problems,"$6,785","$1,332","$19,572","$6,785",...,NaN,NaN,NaN,NaN,NaN,"$6,862","$5,916","$2,535","$7,888",NaN
3,DRG,MS951,MSDRG 951,NaN,NaN,Other Factors Influencing Health Status,"$7,704",$996,"$27,236","$7,704",...,"$11,458","$8,488","$8,476","$9,009","$9,251","$27,236","$23,479","$1,803","$15,958",NaN
4,DRG,MS833,MSDRG 833,NaN,NaN,Other Antepartum Diagnoses Without O.R. Proced...,"$8,369","$1,312","$8,369","$8,369",...,NaN,NaN,NaN,NaN,NaN,"$6,096","$5,255","$3,833","$4,980",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67704,SUP,ZR192,27200020,NaN,0272,"BURR, SHANNON, 2.3 (ZR-192) (ZR192)",$141,$45,$141,$141,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$94,NaN
67705,SUP,ZXR00 240,V2632,NaN,0276,"LENS, IOL, TECNIS, 24 DIOP (ZXR00 24.0) (ZXR00...",$723,$172,$723,$723,...,NaN,NaN,NaN,NaN,NaN,$610,$526,$256,$486,$172
67706,SUP,ZXR00,V2632,NaN,0276,"LENS, IOL, SYMFONY TECNIS, MULTIFOCAL (ZXR00)",$723,$172,$723,$723,...,NaN,NaN,NaN,NaN,NaN,$610,$526,$256,$486,$172
67707,SUP,ZXT150,V2788,NaN,0276,"LENS, IOL, SYMFONY TORIC 150 (ZXT150)",$723,$172,$723,$723,...,NaN,NaN,NaN,NaN,NaN,$610,$526,$256,$486,$172


In [6]:
def payer_category_from_payer(payer):
    payer = payer.strip()
    
    if payer == 'GROSS CHARGE' or payer == 'RECONSTRUCTED CHARGE':
        return 'gross'
    elif payer == 'CASH PRICE':
        return 'cash'
    elif payer == 'DEIDENTIFIED MIN' or payer == 'DE-IDENTIFIED  MINIMUM' or payer == 'Min Price':
        return 'min'
    elif payer == 'DEIDENTIFIED MAX' or payer == 'DE-IDENTIFIED MAXIMUM' or payer == 'Max Price':
        return 'max'
    
    return 'payer'

In [7]:
df_mid = pd.DataFrame(df_in)

df_mid.columns = list(map(lambda c: c.strip(), df_mid.columns.to_list()))
df_mid = df_mid.rename(columns={
    'Procedure': 'local_code',
    'Code Type': 'line_type',
    'Code': 'code',
    'NDC': 'ndc',
    'CPT/HCPCS Code': 'hcpcs_cpt',
    'Beaumont Health Charge Code Description': 'description',
    'Rev Code': 'rev_code',
    'REV CODE': 'rev_code',
    'Procedure Description': 'description',
    'FS ID': 'additional_generic_notes'
})

if 'GROSS CHARGE' in df_mid.columns.to_list():
    money_idx = df_mid.columns.to_list().index('GROSS CHARGE')
elif 'RECONSTRUCTED CHARGE' in df_mid.columns.to_list():
    money_idx = df_mid.columns.to_list().index('RECONSTRUCTED CHARGE')
    
money_columns = df_mid.columns.to_list()[money_idx:]
remaining_columns = df_mid.columns.to_list()[:money_idx]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')

df_mid['description'] = df_mid['description'].str.strip()
df_mid['payer_name'] = df_mid['payer_name'].str.strip()
    
df_mid['standard_charge'] = df_mid['standard_charge'].astype(str)
df_mid['standard_charge'] = df_mid['standard_charge'].apply(cleanup_dollar_value)
df_mid = df_mid[df_mid['standard_charge'] != 'nan']
df_mid = df_mid[df_mid['standard_charge'] != '#VALUE!']
df_mid = df_mid[df_mid['standard_charge'] != '#N/A']
df_mid = df_mid[df_mid['standard_charge'] != '-']
df_mid = df_mid[df_mid['standard_charge'] != '!']
df_mid = df_mid.dropna(subset=['standard_charge'], axis=0)
    
df_mid['additional_payer_specific_notes'] = None
df_mid['contracting_method'] = None
df_mid.loc[df_mid['standard_charge'].str.isalpha(), 'contracting_method'] = 'other'
df_mid.loc[df_mid['standard_charge'].str.isalpha(), 'additional_payer_specific_notes'] = df_mid[df_mid['standard_charge'].str.isalpha()]['standard_charge']
df_mid.loc[df_mid['standard_charge'].str.isalpha(), 'standard_charge'] = None
    
if not 'line_type' in df_mid.columns:
    df_mid['line_type'] = None
    
df_mid['eapg'] = None
df_mid['ms_drg'] = None
    
if 'code' in df_mid.columns:
    df_mid.loc[df_mid['line_type'] == 'DRG', 'ms_drg'] = df_mid[df_mid['line_type'] == 'DRG']['code'].apply(
        lambda code: code.split(" ")[-1]
    )
            
    df_mid['code'] = df_mid['code'].fillna('')
    df_mid['code'] = df_mid['code'].apply(lambda code: str(int(code)) if type(code) == int or type(code) == float else code)

    if not 'hcpcs_cpt' in df_mid.columns:
        df_mid['hcpcs_cpt'] = df_mid['code'].apply(lambda code: code if len(code) == 5 else None)
        
    df_mid.loc[df_mid['code'] == '', 'code'] = None
else:
    df_mid['code'] = None
        
if not 'hcpcs_cpt' in df_mid.columns:
    df_mid['hcpcs_cpt'] = None
    
df_mid = df_mid[df_mid['hcpcs_cpt'] != 'deleted']
        
df_mid['payer_category'] = df_mid['payer_name'].apply(payer_category_from_payer)

df_mid['plan_name'] = None

df_mid['hospital_id'] = ccn
df_mid['apr_drg'] = None
df_mid['modifiers'] = None
df_mid['alt_hcpcs_cpt'] = None
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['billing_class'] = 'facility'
df_mid['standard_charge_percent'] = None

if not 'setting' in df_mid.columns:
    df_mid['setting'] = None

if not 'additional_generic_notes' in df_mid.columns:
    df_mid['additional_generic_notes'] = None

In [8]:
df_mid['eapg'] = df_mid.apply(
    lambda row: None if type(row['eapg']) == str and row['eapg'].startswith('Custom') else row['eapg'], 
    axis=1
)
    
df_mid['hcpcs_cpt'] = df_mid['hcpcs_cpt'].apply(lambda cpt: cpt.replace(" ", "") if type(cpt) == str else cpt)

df_mid['hcpcs_cpt'] = df_mid['hcpcs_cpt'].fillna('')

In [9]:
df_mid[df_mid['hcpcs_cpt'].str.len() > 5][['code', 'description', 'hcpcs_cpt']]

,code,description,hcpcs_cpt


In [10]:
df_mid[df_mid['hcpcs_cpt'].str.len() == 4][['code', 'description', 'hcpcs_cpt']]

,code,description,hcpcs_cpt


In [11]:
df_mid.loc[df_mid['hcpcs_cpt'].str.len() > 5, 'code'] = df_mid[df_mid['hcpcs_cpt'].str.len() > 5]['hcpcs_cpt']
df_mid.loc[df_mid['hcpcs_cpt'].str.len() > 5, 'hcpcs_cpt'] = None
df_mid.loc[df_mid['hcpcs_cpt'].str.len() == 4, 'code'] = df_mid[df_mid['hcpcs_cpt'].str.len() == 4]['hcpcs_cpt']
df_mid.loc[df_mid['hcpcs_cpt'].str.len() == 4, 'hcpcs_cpt'] = None

In [12]:
df_mid.loc[df_mid['additional_generic_notes'] == 'Office Clinic Provider Fee', 'line_type'] = 'office clinic provider fee'
df_mid.loc[df_mid['additional_generic_notes'] == 'Hospital Outpatient Department Provider Fee', 'line_type'] = 'hosp. OP dept. provider fee'

df_mid.loc[df_mid['hcpcs_cpt'] == 'RN001', 'code'] = 'RN001'
df_mid.loc[df_mid['hcpcs_cpt'] == 'RN001', 'hcpcs_cpt'] = None

df_mid.loc[df_mid['hcpcs_cpt'] == '', 'hcpcs_cpt'] = None

if 'local_code' in df_mid.columns:
    df_mid['local_code'] = df_mid['local_code'].str[:40]
else:
    df_mid['local_code'] = None

if not 'rev_code' in df_mid.columns:
    df_mid['rev_code'] = None

df_mid['ndc'] = df_mid['ndc'].fillna('')
df_mid.loc[df_mid['ndc'].str.startswith("WBH"), 'ndc'] = None
df_mid.loc[df_mid['ndc'] == '', 'ndc'] = None

In [13]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,230020,DRG,Normal Newborn,NaN,MS795,MSDRG 795,795,None,None,None,...,facility,None,gross,GROSS CHARGE,None,5630,None,None,None,None
1,230020,DRG,Interstitial Lung Disease Without Cc/McC,NaN,MS198,MSDRG 198,198,None,None,None,...,facility,None,gross,GROSS CHARGE,None,6057,None,None,None,None
2,230020,DRG,Neonate With Other Significant Problems,NaN,MS794,MSDRG 794,794,None,None,None,...,facility,None,gross,GROSS CHARGE,None,6785,None,None,None,None
3,230020,DRG,Other Factors Influencing Health Status,NaN,MS951,MSDRG 951,951,None,None,None,...,facility,None,gross,GROSS CHARGE,None,7704,None,None,None,None
4,230020,DRG,Other Antepartum Diagnoses Without O.R. Proced...,NaN,MS833,MSDRG 833,833,None,None,None,...,facility,None,gross,GROSS CHARGE,None,8369,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2234385,230020,SUP,"PDS FLEXIBLE PLATE, .15 X 50 X 40 (ZFP8)",0278,ZFP8,C1713,None,None,None,C1713,...,facility,None,payer,WC OTHER [1171],None,1095,None,None,None,None
2234393,230020,SUP,"LENS, IOL, TECNIS, 24 DIOP (ZXR00 24.0) (ZXR00...",0276,ZXR00 240,V2632,None,None,None,V2632,...,facility,None,payer,WC OTHER [1171],None,172,None,None,None,None
2234394,230020,SUP,"LENS, IOL, SYMFONY TECNIS, MULTIFOCAL (ZXR00)",0276,ZXR00,V2632,None,None,None,V2632,...,facility,None,payer,WC OTHER [1171],None,172,None,None,None,None
2234395,230020,SUP,"LENS, IOL, SYMFONY TORIC 150 (ZXT150)",0276,ZXT150,V2788,None,None,None,V2788,...,facility,None,payer,WC OTHER [1171],None,172,None,None,None,None


In [14]:
df_out.to_csv("rate_" + ccn + ".csv", index=False)

In [15]:
df_out[df_out['payer_category'] == 'payer']['payer_name'].value_counts()

payer_name
UHC GOLDEN RULE [18065]                       67248
AETNA MICHIGAN PREFERRED [18005]              67248
BCN HMO [1016]                                61282
FIRST HEALTH [18030]                          61137
AETNA VARIPRO [1958]                          61137
PNOA [19069]                                  61137
MULTIPLAN PPO [18050]                         61137
AETNA HMO POS [17005]                         61113
MCLAREN HMO [17020]                           61113
PRIORITY HEALTH HMO EPO POS [17025]           61098
BEAUMONT HEALTH EMPLOYEE PLAN [18010]         60991
HIX PRIORITY BEAUMONT HEALTH [14040]          60986
HAP ALLIANCE PPO EPA EPO [18035]              60912
FARMERS [1810]                                60813
WC OTHER [1171]                               60718
HAP ALLIANCE HMO [17010]                      56765
MMCR ABH AETNA MI HEALTH LINK OON [1963]       2181
MMCR MCLAREN [12055]                           2181
MMCR MOLINA MEDICARE COMPLETE CARE [12042]     2181
M

In [16]:
df_out[df_out['payer_category'] != 'payer']['payer_name'].value_counts()

payer_name
DE-IDENTIFIED  MINIMUM    67669
DE-IDENTIFIED MAXIMUM     67669
GROSS CHARGE              67355
CASH PRICE                67355
Name: count, dtype: int64